In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document


In [ ]:
docs=[
    Document(page_content="Langchain helps build LLM applications"),
    Document(page_content="Pinecone is a vector database for sematic search."),
    Document(page_content="The Eiffel Tower is located in paris"),
     Document(page_content="France is a popular tourist destination."),
      Document(page_content="The Eiffel Tower is a tower"),
]




VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x100782e40>, search_kwargs={})

In [14]:
#Dense Retriever(FAISS+HuggingFace)
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
dense_vectorstore=FAISS.from_documents(docs,embedding_model)
dense_retriever=dense_vectorstore.as_retriever()
dense_retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x100782d80>, search_kwargs={})

In [33]:
#sparse retriever(BM25)

sparse_retriever=BM25Retriever.from_documents(docs)
sparse_retriever.k=3

#comnibe with ensembler retriever
hybrid_retriever=EnsembleRetriever(retrievers=[dense_retriever,sparse_retriever],
weight=[0.2,0.8])
hybrid_retriever


EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x100782d80>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x12f9895e0>, k=3)], weights=[0.5, 0.5])

In [34]:
query="How can I build an Eiffel Tower ?"
results=hybrid_retriever.invoke(query)
for i,doc in enumerate(results):
    print(f"\n Document{i+1}:\n{doc.page_content}")



 Document1:
The Eiffel Tower is a tower

 Document2:
Langchain helps build LLM applications

 Document3:
The Eiffel Tower is located in paris

 Document4:
France is a popular tourist destination.


RAG Pipeline with hybrid Retriever

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
import os
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [36]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Define your prompt
prompt = PromptTemplate.from_template("""
Answer the question based on context below.
Context: {context}
Question: {input}
""")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4)

llm


ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12f972ab0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12f970c20>, root_client=<openai.OpenAI object at 0x12f9898e0>, root_async_client=<openai.AsyncOpenAI object at 0x12f989730>, temperature=0.4, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [37]:
document_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)
retriever_chain=create_retrieval_chain(retriever=hybrid_retriever,combine_docs_chain=document_chain)

In [38]:
query={"input":"How can I build an app using LLMs?"}
response=retriever_chain.invoke(query)

print("Answer:\n",response["answer"])
print("\n Source Documents")
for i,doc in enumerate(response["context"]):
    print(f"\n Doc{i+1}:{doc.page_content}")



Answer:
 You can build an app using LLMs by using Langchain, which helps build LLM applications.

 Source Documents

 Doc1:Langchain helps build LLM applications

 Doc2:The Eiffel Tower is a tower

 Doc3:Pinecone is a vector database for sematic search.

 Doc4:France is a popular tourist destination.

 Doc5:The Eiffel Tower is located in paris


RERANKING